In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import sklearn
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score as rs
from sklearn.metrics import precision_score as ps
from sklearn.metrics import f1_score as fs
from sklearn.metrics import log_loss
encoder = preprocessing.LabelEncoder()

In [2]:
tdata = pd.read_csv('data.csv')

In [3]:
tdata.head()

,id,location,country,gender,age,sym_on,hosp_vis,vis_wuhan,from_wuhan,death,recov,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6
0,1,"Shenzhen, Guangdong",China,male,66.0,1/3/2020,1/11/2020,1,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Shanghai,China,female,56.0,1/15/2020,1/15/2020,0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Zhejiang,China,male,46.0,1/4/2020,1/17/2020,0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Tianjin,China,female,60.0,NaN,NaN,1,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Tianjin,China,male,58.0,NaN,NaN,0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tdata = tdata.drop('id',axis=1)
tdata = tdata.fillna(np.nan,axis=0)
tdata['age'] = tdata['age'].fillna(value=tdata['age'].mean())
tdata['location'] = encoder.fit_transform(tdata['location'].astype(str))
tdata['country'] = encoder.fit_transform(tdata['country'].astype(str))
tdata['gender'] = encoder.fit_transform(tdata['gender'].astype(str))


In [5]:
tdata[['symptom1']] = encoder.fit_transform(tdata['symptom1'].astype(str))
tdata[['symptom2']] = encoder.fit_transform(tdata['symptom2'].astype(str))
tdata[['symptom3']] = encoder.fit_transform(tdata['symptom3'].astype(str))
tdata[['symptom4']] = encoder.fit_transform(tdata['symptom4'].astype(str))
tdata[['symptom5']] = encoder.fit_transform(tdata['symptom5'].astype(str))
tdata[['symptom6']] = encoder.fit_transform(tdata['symptom6'].astype(str))

In [6]:
tdata['sym_on'] = pd.to_datetime(tdata['sym_on'])
tdata['hosp_vis'] = pd.to_datetime(tdata['hosp_vis'])
tdata['sym_on']= tdata['sym_on'].map(dt.datetime.toordinal)
tdata['hosp_vis']= tdata['hosp_vis'].map(dt.datetime.toordinal)
tdata['diff_sym_hos']= tdata['hosp_vis'] - tdata['sym_on']

In [7]:
tdata.isnull().any()

location        False
country         False
gender          False
age             False
sym_on          False
hosp_vis        False
vis_wuhan       False
from_wuhan       True
death           False
recov           False
symptom1        False
symptom2        False
symptom3        False
symptom4        False
symptom5        False
symptom6        False
diff_sym_hos    False
dtype: bool

In [8]:
tdata.head()

,location,country,gender,age,sym_on,hosp_vis,vis_wuhan,from_wuhan,death,recov,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6,diff_sym_hos
0,116,8,1,66.0,737427,737435,1,0.0,0,0,17,31,23,12,3,1,8
1,113,8,0,56.0,737439,737439,0,1.0,0,0,17,31,23,12,3,1,0
2,154,8,1,46.0,737428,737441,0,1.0,0,0,17,31,23,12,3,1,13
3,129,8,0,60.0,1,1,1,0.0,0,0,17,31,23,12,3,1,0
4,129,8,1,58.0,1,1,0,0.0,0,0,17,31,23,12,3,1,0


In [9]:
print(tdata.isna().sum())


location        0
country         0
gender          0
age             0
sym_on          0
hosp_vis        0
vis_wuhan       0
from_wuhan      4
death           0
recov           0
symptom1        0
symptom2        0
symptom3        0
symptom4        0
symptom5        0
symptom6        0
diff_sym_hos    0
dtype: int64


In [10]:
tdata['age'] = tdata['age'].fillna((tdata['age'].mean()))
tdata['from_wuhan'] = tdata['from_wuhan'].fillna((tdata['from_wuhan'].mode()[0]))

In [11]:
from sklearn.metrics import recall_score as rs
from sklearn.metrics import precision_score as ps
from sklearn.metrics import f1_score as fs
from sklearn.metrics import balanced_accuracy_score as bas
from sklearn.metrics import confusion_matrix as cm

In [12]:
X1 = tdata[['location','country','gender','age','vis_wuhan','from_wuhan','symptom1','symptom2','symptom3','symptom4','symptom5','symptom6','diff_sym_hos']]
Y1 = tdata['death']
classifier1 = KNeighborsClassifier()

n_neighbors = [1,2,3,5,7,9,11,13,15]
weights = ['uniform', 'distance']
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
leaf_size = [10,20,30,40,50]
p = [1,2]

params_grid = {'n_neighbors':n_neighbors,'weights':weights,'algorithm':algorithm,'leaf_size':leaf_size, 'p':p}

gridder = GridSearchCV(estimator=classifier1,param_grid=params_grid,n_jobs=-1,cv=5,verbose=5 )
gridder.fit(X1,np.array(Y1).reshape(Y1.shape[0],1))

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 1536 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 2724 tasks      | elapsed:   45.5s
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:   49.1s finished
C:\Python\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [10, 20, 30, 40, 50],
                         'n_neighbors': [1, 2, 3, 5, 7, 9, 11, 13, 15],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=5)

In [19]:
print(gridder.best_estimator_)


KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')


In [20]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')


In [21]:
X = tdata[['location','country','gender','age','vis_wuhan','from_wuhan','symptom1','symptom2','symptom3','symptom4','symptom5','symptom6','diff_sym_hos']]
Y = tdata['death']

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=10)
classifier.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [23]:
pred = classifier.predict(X_test)

recall_knn = rs(Y_test,pred)
precision_knn = ps(Y_test,pred)
f1_knn = fs(Y_test,pred)
ma_knn = classifier.score(X_test,Y_test)

In [24]:
print('*** Evaluation metrics for test dataset ***\n')
print('Recall Score: ',recall_knn)
print('Precision Score: ',precision_knn)
print('F1 Score: ',f1_knn)
print('Accuracy: ',ma_knn)

*** Evaluation metrics for test dataset ***

Recall Score:  0.3333333333333333
Precision Score:  0.6
F1 Score:  0.42857142857142855
Accuracy:  0.9631336405529954
